In [1]:
# github repository for images for training model
!git clone https://github.com/muxspace/facial_expressions.git

Cloning into 'facial_expressions'...
remote: Enumerating objects: 14214, done.
remote: Total 14214 (delta 0), reused 0 (delta 0), pack-reused 14214
Receiving objects: 100% (14214/14214), 239.65 MiB | 4.73 MiB/s, done.
Resolving deltas: 100% (223/223), done.
Updating files: 100% (13996/13996), done.


In [3]:
import csv
data = {}
with open('facial_expressions/data/legend.csv') as f:
  reader = csv.reader(f)
  next(reader)
  for row in reader:
    key = row[2].lower()
    if key in data:
      data[key].append(row[1])
    else:
      data[key] = [row[1]]

In [4]:
# list of facial expressions 
emotion_list = list(data.keys())
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [5]:
import os
os.mkdir('master_data')
os.mkdir('master_data/training')
os.mkdir('master_data/testing')

In [6]:
for emotion in emotion_list:
  os.mkdir(os.path.join('master_data/training', emotion))
  os.mkdir(os.path.join('master_data/testing', emotion))

In [7]:
# dividing training and testing data

from shutil import copyfile
split_size = 0.8

for emotion, images in data.items():
  train_size = int(split_size * len(images))
  train_images = images[:train_size]
  test_images = images[train_size:]

  for image in train_images:
    source = os.path.join('facial_expressions/images', image)
    dest = os.path.join('master_data/training', emotion, image)
    copyfile(source, dest)
  
  for image in train_images:
    source = os.path.join('facial_expressions/images', image)
    dest = os.path.join('master_data/testing', emotion, image)
    copyfile(source, dest)

In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

2023-07-15 13:47:12.944769: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
# creating model
model = tf.keras.models.Sequential([
    Conv2D(16, (3,3), activation='relu', input_shape=(100, 100, 3)),
    MaxPooling2D(2,2),
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(8, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate = 0.01), loss='categorical_crossentropy', metrics=['accuracy']) # accuracy == acc
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 10, 10, 64)       0

In [10]:
train_dir  = 'master_data/training'
test_dir = 'master_data/testing'

train_datagen = ImageDataGenerator(rescale=1.0/255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(100,100),
    class_mode='categorical',
    batch_size=128
)

test_datagen = ImageDataGenerator(rescale=1.0/255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(100,100),
    class_mode='categorical',
    batch_size=128
)

Found 10941 images belonging to 8 classes.
Found 10941 images belonging to 8 classes.


In [11]:
es = EarlyStopping(monitor='val_acc', patience=2, min_delta=0.01)

In [12]:
model.fit(train_generator, epochs=10, verbose=1, validation_data=test_generator, callbacks=[es])

Epoch 1/10
86/86 [==============================] - 33s 380ms/step - loss: 1.2615 - accuracy: 0.4784 - val_loss: 1.1354 - val_accuracy: 0.5018
Epoch 2/10
86/86 [==============================] - 33s 379ms/step - loss: 1.0296 - accuracy: 0.4918 - val_loss: 0.9815 - val_accuracy: 0.5485
Epoch 3/10
86/86 [==============================] - 32s 377ms/step - loss: 0.8473 - accuracy: 0.6575 - val_loss: 0.6835 - val_accuracy: 0.7486
Epoch 4/10
86/86 [==============================] - 34s 393ms/step - loss: 0.6844 - accuracy: 0.7469 - val_loss: 0.6118 - val_accuracy: 0.7760
Epoch 5/10
86/86 [==============================] - 34s 395ms/step - loss: 0.6220 - accuracy: 0.7723 - val_loss: 0.5867 - val_accuracy: 0.7863
Epoch 6/10
86/86 [==============================] - 34s 395ms/step - loss: 0.6110 - accuracy: 0.7809 - val_loss: 0.5965 - val_accuracy: 0.7809
Epoch 7/10
86/86 [==============================] - 34s 394ms/step - loss: 0.5644 - accuracy: 0.8003 - val_loss: 0.5141 - val_accuracy: 0.8141

In [13]:
# saving the model
model.save('facial_emotion_recognition_model.h5')